# TO DO
Change parallel arrays for checking accuracy to data frame instead <br>

BEFORE PUTTING ON GITHUB!!! <br>
Very small step in prediction mesh <br>
Range of k-values to try <br>
File name (not the partial data file) <br>

In [ ]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.gridspec as gridspec
from sklearn import neighbors
import pandas as pd
from pandas import DataFrame, Series
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import operator

# Format data
Indicate the name of the csv file with a column to each parameter, a row to each trial, and both row and column headers. The classification indicator must be the last column and contain either the integer 0 or 1 only.

In [ ]:
inputDataFileName = 'PartialParameterData.csv'

In [ ]:
# Load data into data frame, ignoring the 1st column of trial indices
with open(inputDataFileName) as f:
    ncols = len(f.readline().split(','))

parameterDF = pd.read_csv(inputDataFileName, delimiter=',', index_col=0, header=0 )

In [ ]:
# Delete the indexing column and reset the indices
parameterDF = parameterDF.reset_index()
del parameterDF['index']

In [ ]:
# Normalize the data
params = parameterDF.columns.tolist() # List of labels for indexing
normDF = DataFrame()

for param in params[:-1]: # Normalize each parameter column (not classification column)
    normalizedData = ( parameterDF[param] - parameterDF[param].mean() ) / parameterDF[param].std()
    normDF = pd.concat( [normDF, Series(normalizedData) ], axis=1 )
    normDF.rename( columns={ 0:param } )

# Add the classification column back in
normDF = pd.concat( [normDF, parameterDF[ params[-1] ] ], axis=1 )

In [ ]:
# Group by classification
groups = normDF.groupby( normDF.columns[-1] )
successGroup = groups.get_group(1)
failureGroup = groups.get_group(0)

# Find optimal k and weight method for each pair of parameters

In [ ]:
numParams = len(params)-1 # For looping through parameters

# Set the range of values of k around the typical standard for k=sqrt(n)
# And only look at the odd values for k
kMid = (int)( sqrt(parameterDF.shape[0]) ) # k nearest neighbors

if kMid%2 is 0: # Want odd k
    kMid = kMid + 1
    
kBeg = kMid - 4
kEnd = kMid + 2
ks = np.arange(kBeg, kEnd, 2)

In [ ]:
bestClassifiersByWeight = []
overallBestClassifiers = []
overallBestKs = []
overallBestAccValues = []
overallBestWeights = []

accScoresByWeight = []
allAccScores = [] 

xMins = []
xMaxs = []
yMins = []
yMaxs = []

comparisons = []

maxAccValues = []
bestKs = []
bestClassifiers = []
accScoresByParams = []

# Create list of comparison labels
for xNum in range(numParams):
    for yNum in range(numParams):
        if xNum < yNum:
            comparisons.append( params[xNum]+' vs '+params[yNum] )
            

for rowNum in range(numParams):    

    for colNum in range(numParams): 

        # Now inside the loop for each set of params

        if rowNum < colNum: # Exclude repeated plots 

            # Now inside the loop for each set of params we use (without the ones we don't use)
            print()
            print( '     '+str(params[colNum]) + ' vs ' + str(params[rowNum]) + ':')                

            XTrains = []
            XTests = []
            yTrains = []
            yTests = []

            accuracyScores = [] 
            classifiers = []

            # Get data
            x1 = normDF[ params[colNum] ] # x-axis
            x2 = normDF[ params[rowNum] ] # y-axis
            X = np.column_stack( (x1, x2) )

            # Get overall min and max values for plotting and prediction testing
            xMin, xMax = x1.min(), x1.max() 
            yMin, yMax = x2.min(), x2.max()            
            xMins.append(xMin), xMaxs.append(xMax), yMins.append(yMin), yMaxs.append(yMax)           

            # Split data into training and testing samples
            XTrain, XTest, yTrain, yTest = train_test_split(X, normDF[ params[-1] ])            
            XTrains.append(XTrain), XTests.append(XTest), yTrains.append(yTrain), yTests.append(yTest)            
            # USING SAME DATA FOR TRAINING VS TESTING FOR EACH K THAT WE TRY

            # loop for each k
            for k in ks:

                # Now inside the loop for each k for the set of params

                # Fit the classifier
                classifier = neighbors.KNeighborsClassifier( k ).fit(XTrain, yTrain)
                classifiers.append(classifier)                

                # Predict test data and find accuracy for this k
                predicted = classifier.predict( XTest )
                acc = accuracy_score( yTest, predicted )
                accuracyScores.append( acc )  
                print('          k = '+str(k)+'  accuracy = '+str(acc))

            # Now back to the loop for each set of params that we use

            # Find the best accuracy for this set of params, get the cooresponding k and classifier
            maxAccIndex, maxAccValue = max(enumerate(accuracyScores), key=operator.itemgetter(1))
            bestK = ks[maxAccIndex]
            maxAccValues.append( maxAccValue ) # actual accuracy
            bestKs.append( bestK ) # just to know 
            bestClassifiers.append( classifiers[maxAccIndex] ) # for plotting  
            accScoresByParams.append(accuracyScores)

            print()
            print( '          Max accuracy at k=' + str(bestK) + ' at ' + str(maxAccValue) )

        # Back to the loop for each set of params (with repeats)
    overallBestClassifiers.append( bestClassifiers )


# Plot training data & prediction boundaries with optimized accuracy

# CHANGE STEP SIZE FOR PREDICTION MESH!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1

In [ ]:
h = 0.005  # Step size in the prediction mesh
colMap = ListedColormap( ['#F08080', '#98FB98'] ) # Prediction mesh colors
meshXs = []
meshYs = []
meshPredictions = [] 

# Predict a meshgrid of data points
for num in range( len(bestClassifiers) ):
    meshX, meshY = np.meshgrid(np.arange(xMins[num]-1, xMaxs[num]+1, h),
                               np.arange(yMins[num]-1, yMaxs[num]+1, h))
    meshXs.append(meshX), meshYs.append(meshY)
    meshPredicted = bestClassifiers[num].predict(np.c_[meshX.ravel(), meshY.ravel()])
    meshPredicted = meshPredicted.reshape(meshX.shape)
    meshPredictions.append(meshPredicted)

In [ ]:
# Prepare the figure for plotting
plt.rcParams.update({'figure.autolayout': False})
plt.figure( figsize=(15,15) )
G = gridspec.GridSpec(numParams, numParams)
plt.subplots_adjust( hspace=0.1, wspace=0.1 ) 
plt.rc('font', size=20)
plt.suptitle('Optimized KNN Classifiers', x=0.5, y=0.9)
axarr = []
indices = [] # For indexing row and column labels

for xNum in range(numParams):
    for yNum in range(numParams):
        if xNum < yNum:
            indices.append( (params[xNum],params[yNum]) ) 
            ax = plt.subplot( G[yNum-1, xNum] )
            axarr.append(ax)
            if xNum is 0:
                plt.ylabel( params[yNum] )
            if xNum is not 0:
                plt.yticks( visible=False )
            if yNum is numParams-1:
                plt.xlabel( params[xNum] )
                plt.xticks( rotation=45 )
            if yNum is not numParams-1:
                plt.xticks( visible=False ) 
                         
# Plot 
for num in range(len(indices)):                
    # Plot decision boundary
    axarr[num].pcolormesh(meshXs[num], meshYs[num], meshPredictions[num], cmap=colMap)
    
    # Plot the training data 
    xSuccesses = successGroup[ indices[num][1] ].head(200)
    ySuccesses = successGroup[ indices[num][0] ].head(200)
    axarr[num].scatter( xSuccesses, ySuccesses, marker='o', color='#006400', alpha=0.7, label='success' )
    
    xFailures = failureGroup[ indices[num][1] ].head(200)
    yFailures = failureGroup[ indices[num][0] ].head(200)
    axarr[num].scatter( xFailures, yFailures, marker='s', color='#FF0000', alpha=0.7, label='failure')
            
    axarr[num].set_xlim( -3,3 )
    axarr[num].set_ylim( -3,3 ) 
            
#plt.legend(bbox_to_anchor=(1,1), loc=0, borderaxespad=-4)
#plt.legend(bbox_to_anchor=(1,1), loc=1, borderaxespad=-4)
plt.legend(bbox_to_anchor=(1,1), loc=2, borderaxespad=-11)
plt.savefig('KNNPlotOptimized.png')